In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\korean_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'korean']

In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)  
data

,emd_cd,korean
base_date,,
2018-01-31,50110250,7.209306e+06
2018-02-28,50110250,5.579125e+06
2018-03-31,50110250,7.615021e+06
2018-04-30,50110250,8.498349e+06
2018-05-31,50110250,8.916782e+06
...,...,...
2021-02-28,50130620,4.321265e+06
2021-03-31,50130620,4.744309e+06
2021-04-30,50130620,5.236706e+06


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50130570  ##각행정동코드

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,korean
base_date,
2018-01-31,6.295881e+06
2018-02-28,5.401902e+06
2018-03-31,5.908619e+06
2018-04-30,5.737648e+06
2018-05-31,6.000331e+06
2018-06-30,5.766369e+06
2018-07-31,7.112256e+06
2018-08-31,6.648615e+06
2018-09-30,6.002076e+06


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'korean': base_date
 2018-01-31    6.295881e+06
 2018-02-28    5.401902e+06
 2018-03-31    5.908619e+06
 2018-04-30    5.737648e+06
 2018-05-31    6.000331e+06
 2018-06-30    5.766369e+06
 2018-07-31    7.112256e+06
 2018-08-31    6.648615e+06
 2018-09-30    6.002076e+06
 2018-10-31    6.041744e+06
 2018-11-30    6.069191e+06
 2018-12-31    6.251690e+06
 2019-01-31    6.627001e+06
 2019-02-28    5.823677e+06
 2019-03-31    6.304733e+06
 2019-04-30    6.130456e+06
 2019-05-31    6.453951e+06
 2019-06-30    5.990529e+06
 2019-07-31    6.556196e+06
 2019-08-31    6.448892e+06
 2019-09-30    5.839710e+06
 2019-10-31    6.266654e+06
 2019-11-30    6.080132e+06
 2019-12-31    6.432842e+06
 2020-01-31    6.740625e+06
 2020-02-29    5.117892e+06
 2020-03-31    5.354102e+06
 2020-04-30    5.031930e+06
 2020-05-31    5.533523e+06
 2020-06-30    5.924779e+06
 2020-07-31    6.493685e+06
 2020-08-31    6.372476e+06
 2020-09-30    5.715131e+06
 2020-10-31    6.343063e+06
 2020-11-30    6.603112e+06

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -4.706865
p-value : 0.000082
Critical Values :
	1%: -3.600983
	5%: -2.935135
	10%: -2.605963


In [8]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=1214.661, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=1213.791, Time=0.11 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=1215.404, Time=0.05 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=1216.918, Time=0.05 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=1218.440, Time=0.07 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=1220.363, Time=0.10 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=1213.462, Time=0.03 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=1215.460, Time=0.09 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=1217.266, Time=0.05 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=1218.766, Time=0.20 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=1220.408, Time=0.12 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=1215.494, Time=0.04 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=1217.459, Time=0.19 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=1218.805, Time=0.18 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=1220.534, Time=0.14 sec
 ARIMA(3,0

ARIMA(order=(1, 0, 0), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [9]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.korean.values, order=(1,0,0))   

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(1, 0)   Log Likelihood                -603.727
Method:                       css-mle   S.D. of innovations         422780.410
Date:                Sat, 11 Sep 2021   AIC                           1213.453
Time:                        02:36:17   BIC                           1218.666
Sample:                             0   HQIC                          1215.364
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       6.126e+06   8.85e+04     69.224      0.000    5.95e+06     6.3e+06
ar.L1.y        0.2691      0.147      1.830      0.067      -0.019       0.557
                                    Roots           

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


# 예측하기

In [10]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([6182515.23814861, 6140967.28953096]), array([422780.41031353, 437818.35387097]), array([[5353880.86056502, 7011149.61573219],
       [5282859.08417324, 6999075.49488867]]))
